This is going to make the training dataset with the midi files using Joey's parser. 

In [7]:
import numpy as np
import midi
import glob
import json
from midi_parser import midiToStatematrix

In [31]:
# This method takes in midi file and returns drums/piano as midi files
def get_melody_and_drums(midifile):
    name = midifile.split("/")
    name = name[-1][:-4]
    pattern = midi.read_midifile(midifile)
    
    melody_pattern = midi.Pattern()
    drum_pattern = midi.Pattern()

    melody_pattern.resolution = pattern.resolution
    drum_pattern.resolution = pattern.resolution

    for track in pattern:
        total_ticks = [0] * 17
    
        melody_track = midi.Track()
        drum_track = midi.Track()
    
        melody_channels = [0]
        for i in xrange(len(track)):
            evt = track[i]
         
            if isinstance(evt, midi.ProgramChangeEvent):
                if evt.data in [[0], [1], [2], [4], [5]]:
                    melody_channels.append(evt.channel)
                    melody_track.append(evt)
                
            elif isinstance(evt, midi.TimeSignatureEvent) or \
                    isinstance(evt, midi.SetTempoEvent) or \
                    isinstance(evt, midi.KeySignatureEvent):          
                melody_track.append(evt)
                drum_track.append(evt)
            
            elif isinstance(evt, midi.NoteEvent):
                total_ticks = [x + evt.tick for x in total_ticks]
            
                if (evt.channel == 9):
                    if type(evt) == midi.events.NoteOnEvent:
                        add_event = midi.NoteOnEvent()
                    else:
                        add_event = midi.NoteOffEvent()

                    add_event.tick = total_ticks[9]
                    add_event.channel = evt.channel
                    add_event.data = evt.data

                    drum_track.append(add_event)
                    total_ticks[9] = 0
                
                elif (evt.channel in melody_channels):   
                    if type(evt) == midi.events.NoteOnEvent:
                        add_event = midi.NoteOnEvent()
                    else:
                        add_event = midi.NoteOffEvent()

                    add_event.tick = total_ticks[evt.channel]
                    add_event.channel = evt.channel
                    add_event.data = evt.data

                    melody_track.append(add_event)
                    total_ticks[evt.channel] = 0
        
        eot = midi.EndOfTrackEvent(tick=1)
        drum_track.append(eot)
        melody_track.append(eot)

        if len(melody_track) > 10:
            melody_pattern.append(melody_track)
        if len(drum_track) > 10:
            drum_pattern.append(drum_track)

    # need to write the patterns to midi files
    
    midi.write_midifile("./data/{}.mid".format(name + "_drums"), drum_pattern)
    midi.write_midifile("./data/{}.mid".format(name + "_melody"), melody_pattern)
    

In [32]:
test_file = "songs/CountOnTheBlues.mid"

In [39]:
files = glob.glob('./songs/*.mid')

In [41]:
for song_file in files:
    get_melody_and_drums(song_file)

In [49]:
drum_files = glob.glob('./data/*_drums.mid')
melody_files = glob.glob('./data/*_melody.mid')

In [77]:
drum_matrices = []
for drum_file in drum_files:
    fname = "./data/matrices/" + drum_file.split("/")[-1][:-4] + ".txt"
    mat = midiToStatematrix(drum_file)
    np.savetxt(fname,mat)
for melody_file in melody_files:
    fname = "./data/matrices/" + melody_file.split("/")[-1][:-4] + ".txt"
    mat = midiToStatematrix(melody_file)
    np.savetxt(fname,mat)

In [3]:
drum_mats = glob.glob('./data/matrices/*_drums.txt')
melody_mats = glob.glob('./data/matrices/*_melody.txt')

In [4]:
drum_states = []
for drum_mat in drum_mats:
    temp= np.genfromtxt(drum_mat)
    drum_states.append(temp)
melody_states = []
for melody_mat in melody_mats:
    temp= np.genfromtxt(melody_mat)
    melody_states.append(temp)

In [5]:
np.shape(drum_states)

(247,)

In [8]:
with open("drum_mats.txt",'w') as f:
    json.dump(drum_states,f)

    

TypeError: array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]) is not JSON serializable

In [114]:
JOEY_SIZE = 156

drum_states = []
melody_states = []
for i in range(len(drum_mats[:10])):
    # pad the matrices
    len_drum = np.shape(drum_mats[i])[0]
    len_melody = np.shape(melody_mats[i])[0]
    min_mat = melody_mat
    if(len_drum < len_melody):
        min_mat = drum_mat
    diff = abs(len_drum - len_melody)
    zeros = np.zeros((diff,JOEY_SIZE))
    min_mat = np.concatenate((min_mat,zeros))
    if(len_drum < len_melody):
        drum_mat = min_mat
    else:
        melody_mat = min_mat
    
    # write melody and drum mats
    drum_states.append(drum_mat)
    melody_states.append(melody_mat)
    

IndexError: tuple index out of range

In [115]:
drum_mats[0]

'./data/matrices/2ndMovementOfSinisterFootwear_drums.txt'

In [118]:
len(np.genfromtxt(drum_mats[0]))
len(np.genfromtxt(melody_mats[0]))

2805

This is just to test the padding procedure

In [124]:
NOTE_SIZE = 156

drum_files = glob.glob('./data/*_drums.mid')
melody_files = glob.glob('./data/*_melody.mid')
for i in range(len(drum_files)):
    drum_fname = "./data/matrices/" + drum_files[i].split("/")[-1][:-4] + ".txt"
    melody_fname = "./data/matrices/" + melody_files[i].split("/")[-1][:-4] + ".txt"
    drum_mat = midiToStatematrix(drum_files[i])
    melody_mat = midiToStatematrix(melody_files[i])

    # Pad the matrices to have the same length.

    len_drum = len(drum_mat)
    len_melody = len(melody_mat)
    min_mat = melody_mat
    if(len_drum < len_melody):
        min_mat = drum_mat
    diff = abs(len_drum - len_melody)
    zeros = np.zeros((diff,NOTE_SIZE))
    min_mat = np.concatenate((min_mat,zeros))
    if(len_drum < len_melody):
        drum_mat = min_mat
    else:
        melody_mat = min_mat

    # Write the two matrices
    np.savetxt(drum_fname,drum_mat)
    np.savetxt(melody_fname,melody_mat) 

In [1]:
np.shape(melody_mats)

NameError: name 'np' is not defined